In [137]:
import re
import json
import time
import requests
import pandas as pd
import numpy as np
from functools import reduce
from pandas.io.json import json_normalize

In [158]:
seasons = list(range(2004, 2020))
season_types = ['regular', 'postseason']
total_records = pd.DataFrame()

In [147]:
%%time

result = [i for this_season in seasons for this_season_type in season_types for i in requests.get("https://api.collegefootballdata.com/games", params = {"year": this_season, 'seasonType': this_season_type}).json()]
games = pd.DataFrame(result)

CPU times: user 129 ms, sys: 19.4 ms, total: 149 ms
Wall time: 1.65 s


In [148]:
%%time

result = [i for game_id in games.id.tolist() for i in requests.get("https://api.collegefootballdata.com/games/players", params = {"gameId": game_id}).json()]
stats = pd.DataFrame(result)

CPU times: user 23.7 s, sys: 2.13 s, total: 25.8 s
Wall time: 4min 19s


In [159]:
%%time

result = [i for this_season in seasons for this_season_type in season_types for this_week in games.week.unique().tolist() for i in requests.get("https://api.collegefootballdata.com/games/players",params = {"year": this_season, "week": this_week, 'seasonType': this_season_type}).json()]
stats = pd.DataFrame(result)

CPU times: user 21.3 s, sys: 3.49 s, total: 24.8 s
Wall time: 4min 56s


In [169]:
%%time

result = [i for this_season in seasons for this_season_type in season_types for this_week in games.week.unique().tolist() for i in f2(this_season, this_season_type, this_week)]
stats = pd.DataFrame(result)

         id           name   stat    game.id game.teams.school  \
0    116190  Bryan Randall  14/29  242410259     Virginia Tech   
1    116190  Bryan Randall    153  242410259     Virginia Tech   
2    116190  Bryan Randall    5.3  242410259     Virginia Tech   
3    116190  Bryan Randall      1  242410259     Virginia Tech   
4    116190  Bryan Randall      1  242410259     Virginia Tech   
..      ...            ...    ...        ...               ...   
306  128010    Mike Wafzig     99  242410193        Miami (OH)   
307  128010    Mike Wafzig   49.5  242410193        Miami (OH)   
308  128010    Mike Wafzig      0  242410193        Miami (OH)   
309  128010    Mike Wafzig      0  242410193        Miami (OH)   
310  128010    Mike Wafzig     52  242410193        Miami (OH)   

    game.teams.categories.name game.teams.categories.types.name  season  
0                      passing                            C/ATT    2004  
1                      passing                             

          id               name   stat    game.id game.teams.school  \
0     134713  Michael Greenaway      7  242742005         Air Force   
1     134713  Michael Greenaway    2/2  242742005         Air Force   
2     134713  Michael Greenaway  100.0  242742005         Air Force   
3     134713  Michael Greenaway     28  242742005         Air Force   
4     134713  Michael Greenaway    1/1  242742005         Air Force   
...      ...                ...    ...        ...               ...   
9765  157876         Brett Kern    134  242762649            Toledo   
9766  157876         Brett Kern   44.7  242762649            Toledo   
9767  157876         Brett Kern      0  242762649            Toledo   
9768  157876         Brett Kern      0  242762649            Toledo   
9769  157876         Brett Kern     53  242762649            Toledo   

     game.teams.categories.name game.teams.categories.types.name  season  
0                       kicking                              PTS    2004

          id            name   stat    game.id game.teams.school  \
0     134032   Lester Ricard  18/19  243112655            Tulane   
1     134032   Lester Ricard    323  243112655            Tulane   
2     134032   Lester Ricard   17.0  243112655            Tulane   
3     134032   Lester Ricard      4  243112655            Tulane   
4     134032   Lester Ricard      0  243112655            Tulane   
...      ...             ...    ...        ...               ...   
9277  147623  Andrew Wellock    201  243112199  Eastern Michigan   
9278  147623  Andrew Wellock   40.2  243112199  Eastern Michigan   
9279  147623  Andrew Wellock      0  243112199  Eastern Michigan   
9280  147623  Andrew Wellock      0  243112199  Eastern Michigan   
9281  147623  Andrew Wellock     58  243112199  Eastern Michigan   

     game.teams.categories.name game.teams.categories.types.name  season  
0                       passing                            C/ATT    2004  
1                       passing  

          id        name   stat    game.id game.teams.school  \
0     105712  Scott Hall  15/30  243492572       North Texas   
1     105712  Scott Hall    134  243492572       North Texas   
2     105712  Scott Hall    4.5  243492572       North Texas   
3     105712  Scott Hall      1  243492572       North Texas   
4     105712  Scott Hall      2  243492572       North Texas   
...      ...         ...    ...        ...               ...   
4908  137240  Tom Malone    174  250040030               USC   
4909  137240  Tom Malone   43.5  250040030               USC   
4910  137240  Tom Malone      0  250040030               USC   
4911  137240  Tom Malone      0  250040030               USC   
4912  137240  Tom Malone     56  250040030               USC   

     game.teams.categories.name game.teams.categories.types.name  season  
0                       passing                            C/ATT    2004  
1                       passing                              YDS    2004  
2     

          id         name   stat    game.id game.teams.school  \
0     172902   Zack Asack  12/16  252740150              Duke   
1     172902   Zack Asack    111  252740150              Duke   
2     172902   Zack Asack    6.9  252740150              Duke   
3     172902   Zack Asack      0  252740150              Duke   
4     172902   Zack Asack      1  252740150              Duke   
...      ...          ...    ...        ...               ...   
9681  135795  Cole Chason     69  252740154           Clemson   
9682  135795  Cole Chason   34.5  252740154           Clemson   
9683  135795  Cole Chason      0  252740154           Clemson   
9684  135795  Cole Chason      0  252740154           Clemson   
9685  135795  Cole Chason     35  252740154           Clemson   

     game.teams.categories.name game.teams.categories.types.name  season  
0                       passing                            C/ATT    2005  
1                       passing                              YDS    2

...

            id              name   stat    game.id game.teams.school  \
0      4046678          Ian Book  22/34  401013345        Notre Dame   
1      4046678          Ian Book    343  401013345        Notre Dame   
2      4046678          Ian Book   10.1  401013345        Notre Dame   
3      4046678          Ian Book      2  401013345        Notre Dame   
4      4046678          Ian Book      0  401013345        Notre Dame   
...        ...               ...    ...        ...               ...   
28496  4038144  Corey McCullough    177  401013445      Alcorn State   
28497  4038144  Corey McCullough   35.4  401013445      Alcorn State   
28498  4038144  Corey McCullough      0  401013445      Alcorn State   
28499  4038144  Corey McCullough      3  401013445      Alcorn State   
28500  4038144  Corey McCullough     51  401013445      Alcorn State   

      game.teams.categories.name game.teams.categories.types.name  season  
0                        passing                           

          id              name  stat    game.id game.teams.school  \
0    3916282      Garret Lewis  5/11  401013376              Navy   
1    3916251         Zach Abey   0/5  401013376              Navy   
2    4039425          OJ Davis   0/1  401013376              Navy   
3    3916282      Garret Lewis    81  401013376              Navy   
4    3916251         Zach Abey     0  401013376              Navy   
..       ...               ...   ...        ...               ...   
443  4036929       Ryan Parker     0  401013376              Army   
444  4258457  Javhari Bourdeau     0  401013376              Army   
445  4258521    Landon Salyers     0  401013376              Army   
446  3916807     Mike Reynolds     0  401013376              Army   
447  4036953   Jacob Covington     0  401013376              Army   

    game.teams.categories.name game.teams.categories.types.name  season  
0                      passing                            C/ATT    2018  
1                      

            id            name   stat    game.id game.teams.school  \
0      4371394  Taylor Wallace     42  401117511  Central Arkansas   
1      4371394  Taylor Wallace      0  401117511  Central Arkansas   
2      4371394  Taylor Wallace      0  401117511  Central Arkansas   
3      4371394  Taylor Wallace   36.0  401117511  Central Arkansas   
4      4371394  Taylor Wallace    216  401117511  Central Arkansas   
...        ...             ...    ...        ...               ...   
29483  4027860   Gresch Jensen      0  401121942       Texas State   
29484  4027860   Gresch Jensen      0  401121942       Texas State   
29485  4027860   Gresch Jensen    5.2  401121942       Texas State   
29486  4027860   Gresch Jensen    129  401121942       Texas State   
29487  4027860   Gresch Jensen  16/25  401121942       Texas State   

      game.teams.categories.name game.teams.categories.types.name  season  
0                        punting                             LONG    2019  
1      

            id           name   stat    game.id game.teams.school  \
0      4426768  Jay Bramblett     61  401112227        Notre Dame   
1      4426768  Jay Bramblett      0  401112227        Notre Dame   
2      4426768  Jay Bramblett      0  401112227        Notre Dame   
3      4426768  Jay Bramblett   42.3  401112227        Notre Dame   
4      4426768  Jay Bramblett    423  401112227        Notre Dame   
...        ...            ...    ...        ...               ...   
26996  4035409     Zach Smith      0  401117893             Tulsa   
26997  4035409     Zach Smith      1  401117893             Tulsa   
26998  4035409     Zach Smith    7.7  401117893             Tulsa   
26999  4035409     Zach Smith    309  401117893             Tulsa   
27000  4035409     Zach Smith  23/40  401117893             Tulsa   

      game.teams.categories.name game.teams.categories.types.name  season  
0                        punting                             LONG    2019  
1                  

            id              name   stat    game.id game.teams.school  \
0      4373800     Kwabena Bonsu      0  401114334              Army   
1      4036972   Rod Stoddard II      0  401114334              Army   
2      4036932    Jake Ellington      0  401114334              Army   
3      4373768      Wilson Catoe      0  401114334              Army   
4      4036966        Jack Sides      0  401114334              Army   
...        ...               ...    ...        ...               ...   
29782  4427238    Dillon Gabriel    6.9  401117912               UCF   
29783  4243535  Darriel Mack Jr.     37  401117912               UCF   
29784  4427238    Dillon Gabriel    270  401117912               UCF   
29785  4243535  Darriel Mack Jr.    1/1  401117912               UCF   
29786  4427238    Dillon Gabriel  23/39  401117912               UCF   

      game.teams.categories.name game.teams.categories.types.name  season  
0                      defensive                           

In [168]:
def f2(this_season, this_season_type, this_week):
    response = requests.get("https://api.collegefootballdata.com/games/players",
                                    params = {"year": this_season, "week": this_week, 'seasonType': this_season_type})
    records = json_normalize(
                response.json(),
                ['teams', 'categories', 'types', 'athletes'],
                ['id', ['teams', 'school'], ['teams', 'categories', 'name'], ['teams', 'categories', 'types', 'name']],
                meta_prefix='game.'
    )
    if len(records) == 0:
        return records
    records['season'] = this_season
    print(records)
    records = records[(records.name != ' Team') & 
                      (records['name'] != 'Team') &
                      (records['name'] != '- Team') &
                      (records['stat'] != '--')].reset_index(drop=True)
    
    return records

In [ ]:
results = [i for this_season in seasons for this_season_type in season_types for i in requests.get("https://api.collegefootballdata.com/games", params = {"year": this_season, 'seasonType': this_season_type}).json()]
results = pd.DataFrame(results)
print(len(results))

In [ ]:
for this_season in seasons:
    total_season_records = pd.DataFrame()
    print(this_season)
    for this_season_type in season_types:
        response = requests.get("https://api.collegefootballdata.com/games", params = {"year":this_season, 'seasonType': this_season_type})
        games = pd.read_json(response.text)
        if this_season_type == 'postseason':
            games = games[games.week < 3]
        total_week_records = pd.DataFrame()
        for this_week in games.week.unique():
            response = requests.get("https://api.collegefootballdata.com/games/players",
                                    params = {"year": this_season, "week": this_week, 'seasonType': this_season_type})
            records = json_normalize(
                response.json(),
                ['teams', 'categories', 'types', 'athletes'],
                ['id', ['teams', 'school'], ['teams', 'categories', 'name'], ['teams', 'categories', 'types', 'name']],
                meta_prefix='game.'
            )
            records['season'] = this_season
            records = records[(records.name != ' Team') & 
                              (records['name'] != 'Team') &
                              (records['name'] != '- Team') &
                              (records['stat'] != '--')].reset_index(drop=True)
            total_week_records = total_week_records.append(records)
        total_season_records = total_season_records.append(total_week_records)
    total_records = total_records.append(total_season_records)